# Google Hash Code Challange 2018 - Team Pampamparam

## Qualification round

https://hashcode.withgoogle.com/

### Results

The max score achieved by the winner team is 49,860,860 (by the Pseudo Coders)

This code was not completed on  time, however it is capable of scoring the following values:

| sim           | time          | Total score  |
| ------------- |:-------------:| -----:|
| simualtor1    | 170 sec | 46655900 |
| simualtor2    | 160 sec     | 46554299 |
| simualtor3    | 180 sec   | 47364466|
| simualtor4    | 0 + 0 + 2100 + 750 +1300 sec | 47563568 |
| simualtor5    | 180 sec    | 46979305  |
| simualtor6    | -     | - |

The final result is 48,415,354 points using `sim3` and `sim6` with `ten=10` in less than 20 mins.


### The General Idea

The rides that people ordered are represented as a graph.

* Nodes: rides
* Node i and j is connected IFF one can finish ride i so early that he has time to then go and finish ride j on time as well.
* This ofcourse means, that not all patha in this graph correspond to a viable trip.
* Nevertheless this structure greatly simplifies searching.
* *This can probably be greatly improved*, with this topology a 2 depth maximum search is already infeasible.

We wish to find `n_vachile` node disjunct paths corresponding to viable trips that maximize the reward in total. Clearly this is complicated, possibly requiring a greater graph and some integer programming, so some heuristics is applied to approximate the optimum solution.

Instead of the previous aim we wish to maximize the Reward/Time_spent ratio for each car for either each ride and the road leading there (including waiting) or a series of these. The most simple solution implements the first case.

There is some variability in how the rides could be assigned to cars, main methods are:
* Procceed in a time ordered manner; assign the next task to the car that finished the earliest. (`simulator1`)
* Stick to cars; choose a car, give it new tasks as long as it is not out of time (or the rides are gone), then repeat this for the next car. (`simulator2`)
* Just give tasks to cars cyclically iterating on the ones that did not yet reach `T_time` (upper time limit) (`simulator3`)


**Multiple step optimization:**
* It is possible to not only look for the best value ride for a given car, but to look for the best on a path!
$$w = \frac{Total reward}{Total time}$$
* This is implemented in `simulator4` using $N=2$ as the length of how far ahead we are searching.
* The simlator uses `next_max_2_ride` or `next_max_2_ride2`. The latter
* A generalization would be `next_max_n_long_path_best_out_of_m(self, n, m, rides, bonus)`

Running `next_max_2_ride` on a large dataset is quite impossible, that is why `next_max_2_ride2` is created. The latter version reduces the search space by keeping only the `ten` best first steps. 

Optimizing for 2 steps but then only adding the firs to the car (and setting only this as visited too!) can result in some foresight and hence more optimal paths in total. With `simulator4` and `ten=5` this gives *10048446* --> *10213148* for the metropolis, however when I remove the long rides this result worsens. 

**Parallel optimization:**
We can search for the next one ride for multiple cars at once using maximal bipartite matching. 
This is implemented in `match_finder` and is used in `simulator7` and `simulator8` (for the metropolis).

The results compared to `sim3 (sim6)`: 
* (A same), 
* (B *176520* --> *175452*, 10 sec), 
* (C *15791852* --> *15794209*, 15 ksec), 
* (D *10981305* --> *11037466*, 13,5 hours), 

I am sure one can do way better in this much time. And all in all, the difference is not substantial, so although this could be optimized this is probably not the solution leading to the 50Mpoints.


**In depth optimization**
Another idea is to build a more complex graph, one in which it makes to calculate a longest possible path for a car, then remove the corresponding rides and iterate this process.


**Other ideas:**
* It is worth to try to uniformly scatter some of the cars before we start assigning rides. This is done in `simulator5`. It would be even smarter to somehow modify the weight function in a way that cars gravitate to their assigned area or repell each other a bit.

#### Metropolis as special case:
Our problems originate from the metropolis input file. There the roads leading out of the city deceive cars that can forsee only a single ride, and leads them out of the city.



<img src="simulator3_metropolis.png" />

To some extent this is taken care of in `simulator6`, where I modify long paths (that are likely to lead out of the city) to only keep the ones that can be performed colse to `T_time` and for these I move `t_min` for them to get executed close to the end.

With a simple `next_ride()` this brings an improvement of $10^6$ (to *10793180*) and a 40% decreased runtime (100 sec, params 0.7, 3.0), while for `next_ride2()` with `ten=10` the result is *10960929* in 800 sec. 
If I only add the first element of the path:  *10960929*--> *10939117* (`ten=5`) and *10981305* --> *10917184* (`ten=10`).

In [1]:
"""
Import libraries
"""

import numpy as np
import scipy as sp
#import networkx as nx #only used for bipartite matching
import os, sys
import math, time

from operator import itemgetter, attrgetter

#for progressbars
#from tqdm import tqdm

### The weights for optimization

* `value_of_edge` and `value_of_first` give the single ride weights
* `value_of_path` and `value_of_first_path` are self explanatory
* It would be nice to have something that promotes the scattering of the vechiles.

In [2]:
def value_of_edge(ride1,ride2, bonus):
    
    Time_spent  = 0
    Money_earnt = ride2.distance
    
    d_endpoints  = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
    
    if d_endpoints + ride1.t_finish < ride2.t_min:
        Time_spent = ride2.t_min - ride1.t_finish + ride2.distance
    else:
        Time_spent =  ride2.distance + d_endpoints
        
        
    if d_endpoints + ride1.t_finish <= ride2.t_min:
        Money_earnt += bonus
    
    return  float(Money_earnt) / Time_spent

def value_of_path(ride_list, bonus):

    Time_spent  = 0
    Money_earnt = 0
    
    ride1 = ride_list[0]
    for i in range(1,len(ride_list)):
        ride2 = ride_list[i]
        
        d_endpoints  = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
        
        if d_endpoints + ride1.t_finish < ride2.t_min:
            Time_spent += ride2.t_min - ride1.t_finish + ride2.distance
        else:
            Time_spent +=  ride2.distance + d_endpoints
        
        if d_endpoints + ride1.t_finish <= ride2.t_min:
            Money_earnt += bonus + ride2.distance
        else:
            Money_earnt += ride2.distance
        
        ride1 = ride2   
    
    return  float(Money_earnt) / Time_spent

def value_of_first(ride2, bonus):
    
    Time_spent  = 0
    Money_earnt = ride2.distance
    
    d_endpoints  = ride2.start[0] +  ride2.start[1]
    
    if d_endpoints < ride2.t_min:
        Time_spent = ride2.t_min  + ride2.distance
    else:
        Time_spent =  ride2.distance + d_endpoints
        
        
    if d_endpoints  <= ride2.t_min:
        Money_earnt += bonus
    
    return  float(Money_earnt) / Time_spent


def value_of_first_path(ride_list, bonus):

    zero = Ride(-1, 0,0,  0,0, 0, 0 )
    
    ride_list = [zero] + ride_list
    
    return value_of_path(ride_list, bonus)


### Classes

#### Ride

Rides, besides the obvious data, also point to the ids of the rides that have an apriory chance to come after them.
They also contain information on *when a ride has actually began* and *end*.

The important methods are the `set_outgoing_edges`, and the ones that choose the next optimal ride(s) after this one (which can be done if we know when this ride has been completed). 

After a ride is assigned to a car we set 
```python
self.visited = True
```
and this ride is not considered anymore as part of the graph. We also set `t_start` and `t_finish` to be able to iterate the next ride finder methods iteratively.


#### Car

Each car has a list of rides that are assigned to it and a `no_more_jobs` flag that is set True if the next ride finder method is not able to find a new ride for some reason.
The car also has an `id`, but it is unused.


In [3]:
"""
Main objects 
"""


class Ride(object):
    
    def __init__(self, id_, a,b,x,y,s,f):
        
        self.id    = id_
        
        self.start = [a,b]
        self.finish= [x,y]
        self.t_min = s
        self.t_max = f
        
        self.distance = abs(x-a) + abs(b-y)
        
        self.start_the_latest    = self.t_max - self.distance -1
        self.finish_the_earliest = self.t_min + self.distance ##################################
        
        #start and finish of actual ride
        self.t_start =s
        self.t_finish=s + self.distance  ######################################################## 
        
        self.visited = False
        
        self.incoming_edges = []
        self.outgoing_edges = []
        
    def __str__(self):
        return str(self.start) + " " + str(self.t_min) + " --> " + str(self.finish) + " " + str(self.t_max)
        
    def add_incoming_edge(self, id_):
        
        self.incoming_edges += [id_]
        
    def add_outgoing_edge(self, id_):
        
        self.outgoing_edges += [id_]
        
    def set_outgoing_edges(self, rides):
        for ride in rides:
            if self.finish_the_earliest < ride.start_the_latest and self.id != ride.id:
                self.outgoing_edges += [ride.id]
                
    def set_visited(self):
        self.visited=True
        
    def next_ride(self, rides, bonus):
        max_value = 0
        next_ride = 0
        for i in self.outgoing_edges:
            ride = rides[i]
            d_endpoints = abs(ride.start[1] - self.finish[1]) + abs(ride.start[0] - self.finish[0])
            if self.t_finish + d_endpoints <= ride.start_the_latest and not ride.visited:
                if value_of_edge(self, ride, bonus) > max_value:
                    max_value = value_of_edge(self, ride, bonus)
                    next_ride = ride
                    if d_endpoints + self.t_finish < ride.t_min:
                        # self.t_start = self.t_start
                        pass
                    else:
                        next_ride.t_start =  self.t_finish + d_endpoints
                        next_ride.t_finish=  next_ride.t_start + next_ride.distance
        if next_ride != 0:
            next_ride.set_visited()
            #print next_ride, next_ride.t_finish, value_of_first(next_ride, bonus)
        return next_ride
    
    def next_max_2_ride(self, rides, bonus):
        
        max_value = 0
        next_ride_list = []
        temp_ride_walks= []
        ride_walks     = []
        
        if len(self.outgoing_edges)==0:
            return 0
        #if can --> generate paths
        for i in self.outgoing_edges:
            ride1 = self
            ride2 = rides[i]
            d_endpoints  = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
            if ride1.t_finish + d_endpoints <= ride2.start_the_latest and not ride2.visited:
                
                ride2_t_finish = 0
                if ride1.t_finish + d_endpoints > ride2.t_min:
                    ride2_t_finish = ride1.t_finish + d_endpoints + ride2.distance
                else:
                    ride2_t_finish= ride2.t_min + ride2.distance
                no_triple_flag = True
                for i2 in ride2.outgoing_edges:
                    ride3 = rides[i2]
                    d_endpoints  = abs(ride2.finish[0] - ride3.start[0]) + abs(ride2.finish[1] - ride3.start[1])
                    if ride2_t_finish + d_endpoints <= ride3.start_the_latest and not ride3.visited:
                        ride_walks += [[ride1, ride2,ride3]]
                        no_triple_flag = False
                
                if no_triple_flag:
                    ride_walks += [[ride1, ride2]]
        
        #choose optimum path
        values = [value_of_path(x,bonus) for x in ride_walks]
        if len(values)>0:
            next_ride_list = ride_walks[values.index(max(values))]
            next_ride1 = next_ride_list[0]
            time_finish = next_ride1.t_finish
            for next_ride2 in next_ride_list[1:]:
                #set visited
                rides[next_ride2.id].visited = True
                #set t_start t_finish
                
                d_endpoints = abs(next_ride1.finish[0] - next_ride2.start[0]) + abs(next_ride1.finish[1] - next_ride2.start[1])
                if time_finish + d_endpoints < next_ride2.t_min:
                    time_finish = next_ride2.t_min + next_ride2.distance
                else:
                    time_finish += d_endpoints + next_ride2.distance
                    
                next_ride2.t_finish = time_finish 
                
                next_ride1 = next_ride2
                
                
                
            return next_ride_list[1:]
        else:
            return 0
    def next_max_2_ride2(self, rides, bonus):
        
        next_ride_list = []
        temp_ride_walks= []
        ride_walks     = []
        
        if len(self.outgoing_edges)==0:
            return 0
        #if can --> generate paths
        for i in self.outgoing_edges:
            ride1 = self
            ride2 = rides[i]
            d_endpoints  = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
            if ride1.t_finish + d_endpoints <= ride2.start_the_latest and not ride2.visited:
                temp_ride_walks += [[ride1, ride2]]
        
        #choose at most 10 best
        #ten_best = ride_walks[:]
        #"""
        ten = 10
        ten_best = []
        if len(temp_ride_walks)<=ten:
            ten_best = temp_ride_walks[:]
        else:
            weights = [value_of_path(x,bonus) for x in temp_ride_walks]
            for i1 in range(ten):
                w = max(weights)
                ten_best += [temp_ride_walks[weights.index(w)]]
                weights[weights.index(w)] = -100
        #""" 
        
        temp_ride_walks = []
        
        for r_ in ten_best:
            ride1 = r_[0]
            ride2 = r_[1]
            
            ride2_t_finish = 0
            d_endpoints  = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
            if ride1.t_finish + d_endpoints > ride2.t_min:
                ride2_t_finish = ride1.t_finish + d_endpoints + ride2.distance
            else:
                ride2_t_finish= ride2.t_min + ride2.distance
            
            
            no_triple_flag = True
            
            for i2 in ride2.outgoing_edges:
                ride3 = rides[i2]
                d_endpoints  = abs(ride2.finish[0] - ride3.start[0]) + abs(ride2.finish[1] - ride3.start[1])
                if ride2_t_finish + d_endpoints <= ride3.start_the_latest and not ride3.visited:
                    ride_walks += [[ride1, ride2,ride3]]
                    no_triple_flag = False
            if no_triple_flag:
                ride_walks += [[ride1, ride2]]
        
        #choose optimum path
        values = [value_of_path(x,bonus) for x in ride_walks]
        if len(values)>0:
            next_ride_list = ride_walks[values.index(max(values))]
            next_ride1 = next_ride_list[0]
            time_finish = next_ride1.t_finish
            for next_ride2 in next_ride_list[1:]:
                #set visited
                rides[next_ride2.id].visited = True
                #set t_start t_finish
                
                d_endpoints = abs(next_ride1.finish[0] - next_ride2.start[0]) + abs(next_ride1.finish[1] - next_ride2.start[1])
                if time_finish + d_endpoints < next_ride2.t_min:
                    time_finish = next_ride2.t_min + next_ride2.distance
                else:
                    time_finish += d_endpoints + next_ride2.distance
                    
                next_ride2.t_finish = time_finish 
                
                next_ride1 = next_ride2
                
                
                
            return next_ride_list[1:]
        else:
            return 0
    
    
    
    def next_max_n_long_path_best_out_of_m(self, n, m, rides, bonus):
        """
        Does not work yet
        """
        next_ride_list = []
        ride_walks     = []
        
        if len(self.outgoing_edges)==0:
            return 0
        
        #if can --> generate paths
        no_neighbours = False
        depth = 0
        ride1 = self
        ride1_t_finish = self.t_finish
        while depth<n or no_neighbours:
            
            depth += 1
        
            #collect good neighbours
            for i in self.outgoing_edges:
            
                ride2 = rides[i]
                d_endpoints  = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
                if ride1_t_finish + d_endpoints <= ride2.start_the_latest and not ride2.visited:
                    ride_walks += [[ride1, ride2]]
            #select best m neighbours
            m_best = []
            if len(ride_walks)<m:
                m_best = ride_walks[:]
            else:
                weights = [value_of_path(x,bonus) for x in ride_walks]
                for i1 in range(m):
                    w = max(weights)
                    m_best += [ride_walks[weights.index(w)]]
                    weights[weights.index(w)] = -100
            #we have the new neibours we want to discover
                    
        
        return 0
##########################################################################################################               
def very_first_ride(rides, bonus):
    max_value = 0
    next_ride = 0
    for ride in rides:
        d_endpoints = ride.start[1] + ride.start[0] 
        if d_endpoints <= ride.start_the_latest and not ride.visited:
            if value_of_first(ride, bonus) > max_value:
                max_value = value_of_first(ride, bonus)
                next_ride = ride
                if d_endpoints < ride.t_min:
                    pass
                else:
                    next_ride.t_start = d_endpoints
                    next_ride.t_finish= next_ride.t_start + next_ride.distance
    if next_ride != 0:
        next_ride.set_visited()
        #print next_ride, next_ride.t_finish, value_of_first(next_ride, bonus)
    return next_ride
        
    
def match_finder(cars, rides, bonus):
    car_to_next_ride_dictionary = {}
    
    #build graph
    G=nx.Graph()
    
    last_rides = [ car.rides[-1] for car in cars]
    graph_id_to_car_id = {} 
    
    for i1 in range(len(last_rides)):
        last_ride = last_rides[i1]
        G.add_node(last_ride.id)
        graph_id_to_car_id[last_ride.id] = i1
        
    last_ride_ids = list(graph_id_to_car_id.keys())
        
    for last_ride in last_rides:
        for i1 in last_ride.outgoing_edges:
            d_endpoints  = abs(last_ride.finish[0] - rides[i1].start[0]) + abs(last_ride.finish[1] - rides[i1].start[1])
            if last_ride.t_finish + d_endpoints <= rides[i1].start_the_latest and not rides[i1].visited:
                if not rides[i1].id in G:
                    G.add_node(rides[i1].id)
                G.add_edge(last_ride.id,i1, weight = value_of_edge(last_ride, rides[i1], bonus))
                
    #graph is built
    #max pairing
    mate = nx.max_weight_matching(G, maxcardinality=False)
    
    #make return
    for k,v in mate:
        if k in last_ride_ids:
            rides[v].visited = True
            d_endpoints = abs(rides[k].finish[0] - rides[v].start[0]) + abs(rides[k].finish[1] - rides[v].start[1])
            if rides[k].t_finish + d_endpoints < rides[v].t_min:
                rides[v].t_finish = rides[v].t_min + rides[v].distance
            else:
                rides[v].t_finish = rides[k].t_finish + d_endpoints + rides[v].distance
            
            car_to_next_ride_dictionary[graph_id_to_car_id[k]] = rides[v]
        else:
            rides[k].visited = True
            d_endpoints = abs(rides[v].finish[0] - rides[k].start[0]) + abs(rides[v].finish[1] - rides[k].start[1])
            if rides[v].t_finish + d_endpoints < rides[k].t_min:
                rides[k].t_finish = rides[k].t_min + rides[k].distance
            else:
                rides[k].t_finish = rides[v].t_finish + d_endpoints + rides[k].distance
            
            car_to_next_ride_dictionary[graph_id_to_car_id[v]] = rides[k]
    
    """
    parent_to_children_plus_weights = {}
    childern = []
    
    last_rides = [ car.rides[-1] for car in cars]
    
    for last_ride in last_rides:
        parent_to_children_plus_weights['source'] = [last_ride.id , 1000000, 0] # id, capacity, flow
        
    
    for last_ride in last_rides:
        for i1 in last_ride.outgoing_edges:
            d_endpoints  = abs(last_ride.finish[0] - rides[i1].start[0]) + abs(last_ride.finish[1] - rides[i1].start[1])
            if last_ride.t_finish + d_endpoints <= rides[i1].start_the_latest and not rides[i1].visited:
                
                parent_to_children_plus_weights[last_ride.id] += [[rides[i1].id , value_of_edge(last_ride, rides[i1], bonus), 0]]
                children += [rides[i1].id]
    for child in children:
        parent_to_children_plus_weights[child.id] = ['sink', 1000000, 0]
    del children
    
    #graph created
    #start Ford-Fulkerson
        #start with zero flow
        #while there is a path from source --> sink flow stuff on it
        #when done, collect edges with the maximal capacity --> cut
    
    
    #done Ford-Fulkerson
    """

    return car_to_next_ride_dictionary
    

    
##########################################################################################################
class Car(object):
    
    def __init__(self, id_):
        
        self.id = id_
                
        self.rides= []
        self.no_more_jobs = False
              
    def is_free(self, time):
        
        return_bool  = True
        if len(self.rides)!=0:
            return_bool = time >= self.rides[-1].t_finish
        return return_bool
    
    def get_finish_time(self):
        return self.rides[-1].t_finish
        
    
    def add_ride(self, ride, t_start_):
        
        ride.t_start = t_start
        self.rides+=[ride]   
        
    def next_ride(self, rides, bonus):
        my_next_ride = self.rides[-1].next_ride(rides,bonus)
        
        if my_next_ride == 0:
            self.no_more_jobs = True
        else:
            self.rides.append(my_next_ride)
            
    def next_ride2(self, rides, bonus):
        """
        Doesnt really work yet
        """
        my_next_ride_list = self.rides[-1].next_max_2_ride2(rides,bonus)
        
        if my_next_ride_list == 0:
            self.no_more_jobs = True
        else:
            for my_next_ride in my_next_ride_list:
                self.rides.append(my_next_ride)
                
        #if self.id==1:
        #    if my_next_ride_list != 0:
        #        print 'fill car1:',len(my_next_ride_list)
        #    else: 
        #        print 'fill car1: zero',
        
    def first_ride(self, rides, bonus):
        my_next_ride = very_first_ride(rides, bonus)
            
        if my_next_ride == 0:
            self.no_more_jobs = True
            #print "jajj, elo korben!"
        else:
            self.rides.append(my_next_ride)
    
    
    
    

### Input, Parsing and Output

In [4]:
"""
Reading input files. We will load them into objects in the second function.
"""

def read_input_file(input_name_1):
    list_to_return = []
    
    input_f        = open("./input/" + input_name_1, 'r')

    for line in input_f:
        # do something here if you must
        list_to_return += [line]       
    
    return list_to_return

def load_input_into_objects(input_list):
    
    parameters_line = input_list[0]
    
    parameters = [int(_) for _ in parameters_line.split(" ")]
    n_rows, n_cols, n_vechiles, n_rides, bonus, T_time      = parameters
    
    rides = []
    
    for i in range(1,1+n_rides):
        a,b,x,y,s,f = [int(_) for _ in input_list[i].split(" ")]
        temp_ride = Ride(i-1, a,b,x,y,s,f )
        
        rides += [temp_ride]
        
    return [n_rows, n_cols,rides, bonus, T_time, n_vechiles] 
    

In [5]:
"""
Writing output file
"""

def write_output_file(file_name, cars):
    f = open("./output/" + file_name, "w")
    for car in cars:
        line = [ len(car.rides), ]
        for ride in car.rides:
            line += [ride.id]
        #print ' '.join([int(l) for l in line]) + "\n"
        f.write(' '.join([str(l) for l in line]) + "\n")
    f.close()

### Distributing workload amongst cars

In [6]:
def s_list_insert(list_):
    """
    take the first list element and insert it into the list if the rest is sorted
    the list consists of len 3 tiny lists, sorting is according to the last element
    """
    first = list_[0]
    new_list = list_[1:len(list_)]
    for i in range(0, len(new_list)):
        if first[2] > new_list[i][2]:
            new_list.insert(i+1, first)
            break
        else:
            pass
    return new_list



def simulator1(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    Assign tasks in a timely manner
    """
    
    car_list = []
    car_done = []
    
    for i in range(n_vechiles):
        temp_car = Car(i)
        car_list += [[i, temp_car, 0]]
          
    for car in car_list:
        car[1].first_ride(rides, bonus)
        # set finished time
        car[2]=car[1].get_finish_time()
    
    sorted(car_list, key=itemgetter(2))
        
    while len(car_list)>0:
           
        car_list[0][1].next_ride(rides,bonus)
        
        if car_list[0][1].no_more_jobs:
            #print "no more jobs :("
            car_done += [car_list[0][1]]
            del car_list[0]
        else:
            car_list[0][2]=car_list[0][1].get_finish_time()
            s_list_insert(car_list)      
      
    return car_done
        
        
def simulator2(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    Fill a car with tasks, than do the same to the next car
    """
    
    car_list = []
    car_done = []
    
    for i in range(n_vechiles):
        temp_car = Car(i)
        car_list += [temp_car]
          
    for car in car_list:
        car.first_ride(rides, bonus)
        
        while not car.no_more_jobs:
           
            car.next_ride(rides,bonus)
        
        car_done += [car]         
      
    return car_done
        
def simulator3(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    Assign tasks cyclically
    """
    car_list = []
    car_done = []
    
    for i in range(n_vechiles):
        temp_car = Car(i)
        temp_car.first_ride(rides, bonus)
        car_list += [temp_car]
       
    counter = 0
    while len(car_list)>0:
        
        counter = (counter + 1)%len(car_list)
           
        car_list[counter].next_ride(rides,bonus)
        
        if car_list[counter].no_more_jobs:
            #print "no more jobs :("
            car_done += [car_list[counter]]
            del car_list[counter]    
      
    return car_done

def simulator4(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    2 depth search
    """
    
    car_list = []
    car_done = []
    
    for i in range(n_vechiles):
        temp_car = Car(i)
        temp_car.first_ride(rides, bonus)
        car_list += [temp_car]
       
    counter = 0
    while len(car_list)>0:
        
        counter = (counter + 1)%len(car_list)
           
        car_list[counter].next_ride2(rides,bonus)
        
        #if car_list[counter].id == 1:
        #    print 'car1: ', [(r.id, r.visited) for r in car_list[counter].rides]
        
        if car_list[counter].no_more_jobs:
            #print "no more jobs :("
            car_done += [car_list[counter]]
            del car_list[counter]    
      
    return car_done
    
        
def simulator5(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    distribute some of the cars in a grid-like manner, then procceed as sim3
    """
    
    car_list = []
    car_done = []
    
    for i in range(n_vechiles):
        temp_car = Car(i)
        car_list += [temp_car]
        
    #let's distribute half of the cars in the first round somehow evenly
    n_half_cars = int(n_vechiles *0.9)
    
    #distribute 
    Area_for_a_car = X_len * Y_len / n_half_cars
    lattice_l      = int(np.sqrt(Area_for_a_car))
    
    car_counter = -1
    for x in range(0,X_len,lattice_l ):
        for y in range(0,Y_len,lattice_l ):
            temp_virtual_ride = Ride('virtual', x,y,x,y,x+y,x+y)
            temp_virtual_ride.t_finish = x+y
            temp_virtual_ride.set_outgoing_edges(rides)
            car_counter += 1
            car_list[car_counter].rides += [temp_virtual_ride]
            #print car_list[car_counter].rides[0]
            car_list[car_counter].next_ride(rides,bonus)
            #print car_list[car_counter].rides
            car_list[car_counter].rides = [car_list[car_counter].rides[1]]
            
    
    #normal
    for i in range(car_counter+1, n_vechiles):
        car_list[i].first_ride(rides, bonus)
    
       
    counter = 0
    while len(car_list)>0:
        
        counter = (counter + 1)%len(car_list)
           
        car_list[counter].next_ride(rides,bonus)
        
        if car_list[counter].no_more_jobs:
            #print "no more jobs :("
            car_done += [car_list[counter]]
            del car_list[counter]
      
    return car_done     
        
def simulator6(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    Modify very long rides, then procceed like sim3 or sim4
    """
    
    car_list = []
    car_done = []
    
    for ride in rides:
        if ride.distance > (float(X_len) * 0.7):
            if ride.finish_the_earliest + ride.distance / 3.0 > T_time:
                pass
                ride.t_min = max([ride.t_max - ride.distance, ride.t_min])
                ride.start_the_latest = ride.t_min
            else:
                ride.visited = True
            
    for i in range(n_vechiles):
        temp_car = Car(i)
        temp_car.first_ride(rides, bonus)
        car_list += [temp_car]
        
    
       
    counter = 0
    while len(car_list)>0:
        
        counter = (counter + 1)%len(car_list)
           
        car_list[counter].next_ride2(rides,bonus)
        
        if car_list[counter].no_more_jobs:
            #print "no more jobs :("
            car_done += [car_list[counter]]
            del car_list[counter]    
      
    return car_done


def simulator7(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    Bipartite graph matching for each layer
    """
    car_list = []
    car_done = []
    
            
    for i in range(n_vechiles):
        temp_car = Car(i)
        temp_car.first_ride(rides, bonus)
        car_list += [temp_car]
        
    
    while True:
        
        car_id_to_next_ride_id = match_finder(car_list, rides, bonus)
        
        if len(car_id_to_next_ride_id)==0:
            return car_list
        else:
            for i in car_id_to_next_ride_id.keys():
                
                car_list[i].rides += [car_id_to_next_ride_id[i]]
                
def simulator8(n_vechiles, T_time, bonus, rides, X_len, Y_len):
    """
    Take care of long rides, then do bipartite matching
    """
    car_list = []
    car_done = []
    
    for ride in rides:
        if ride.distance > (float(X_len) * 0.7):
            if ride.finish_the_earliest + ride.distance / 3.0 > T_time:
                pass
                ride.t_min = max([ride.t_max - ride.distance, ride.t_min])
                ride.start_the_latest = ride.t_min
            else:
                ride.visited = True
    
            
    for i in range(n_vechiles):
        temp_car = Car(i)
        temp_car.first_ride(rides, bonus)
        car_list += [temp_car]
        
    
    while True:
        
        car_id_to_next_ride_id = match_finder(car_list, rides, bonus)
        
        if len(car_id_to_next_ride_id)==0:
            return car_list
        else:
            for i in car_id_to_next_ride_id.keys():
                
                car_list[i].rides += [car_id_to_next_ride_id[i]]
    

### Scoring and checking my solutions

In [7]:
"""
Score for self check
"""

def score_solution(rides, cars, bonus, T_time):
    
    score = 0
    used_rides = []
    
    for car in cars:
        time_ = 0
        ride_list = car.rides
        if len(ride_list)>0:
            ride1 = ride_list[0]
            if ride1.id in used_rides:
                print "Error reuse"
            else:
                used_rides += [ride1.id]
            #can I get there?
            if ride1.start[0]+ride1.start[1]>ride1.start_the_latest:
                print "Error1       ",car.id 
            #on time? 
            if ride1.start[0]+ride1.start[1]<=ride1.t_min:
                #print "bonus1"
                score += bonus + ride1.distance 
                time_ = ride1.t_min + ride1.distance -1 #####################################################
            else:
                score += ride1.distance
                time_ += ride1.distance + ride1.start[0]+ride1.start[1] -1 ##################################
            if time_ > ride1.t_max:
                    print "Error overtime 1"
            

            for i in range(1,len(ride_list)):
                ride2 = ride_list[i]
                if ride2.id in used_rides:
                    print "Error reuse"
                else:
                    used_rides += [ride2.id]
                
                #can I get there?
                d_endpoints = abs(ride1.finish[0] - ride2.start[0]) + abs(ride1.finish[1] - ride2.start[1])
                if d_endpoints + time_ > ride2.start_the_latest:
                    print "Error2     ", car.id,  time_ , ride2.start_the_latest, ride2
                    print [(r.id, r.visited) for r in ride_list]
                #on time? 
                if d_endpoints + time_ <= ride2.t_min:
                    score += bonus + ride2.distance
                    time_  = ride2.t_min + ride2.distance
                else:
                    score += ride2.distance
                    time_ += d_endpoints + ride2.distance
                if time_ > ride2.t_max:
                    print "Error overtime 2"
                #print ride2.t_finish, time_
                  
                if time_ > T_time:
                    print "Error3", car.id, time_
                    
                
                ride1 = ride2
        
        
        
    print 'score:', score
    return score

## Putting it all together

In [8]:
"""
MAIN FUNCTION
"""

def main():
    start = time.time()
    
    input_list = [
        'a_example.in',
        'b_should_be_easy.in',
        'c_no_hurry.in',
        'd_metropolis.in',
        'e_high_bonus.in',
    ]
    
    score = 0
    tick_tack =0
    
    for file_name in input_list[0:2]:
        
        tick_tack = time.time()
        
        X_len, Y_len, rides, bonus, T_time, n_vechiles= load_input_into_objects(read_input_file(file_name))
        
        
        for ride in rides:
            ride.set_outgoing_edges(rides)
        
        if file_name != 'd_metropolis.in':
            cars = simulator3(n_vechiles, T_time, bonus, rides, X_len, Y_len)
        else:
            cars = simulator6(n_vechiles, T_time, bonus, rides, X_len, Y_len)

        score += score_solution(rides, cars, bonus, T_time)
            
        write_output_file(file_name[:-3] + '_out1.txt', cars)
        
        print 't:', time.time()-tick_tack
       
        
    end   = time.time()
    print score, "alma", end-start

In [9]:
"""
Perform calculations
"""

if __name__ == '__main__':
    main()

score: 10
t: 0.000575065612793
score: 176520
t: 0.0933971405029
176530 alma 0.0940411090851
